In [1]:
from transformers import pipeline

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Carregar modelo BERTimbau pré-treinado para Q&A
qa_pipeline = pipeline(
    "question-answering",
    model="neuralmind/bert-base-portuguese-cased",
    tokenizer="neuralmind/bert-base-portuguese-cased"
)

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucas\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForQuestionAnswering were not initialized from the model

In [5]:
# Exemplo com documento longo
import pdfplumber
with pdfplumber.open("arquivo.pdf") as pdf:
    documento = " ".join([page.extract_text() for page in pdf.pages])  # Seu documento aqui
pergunta = "Qual é o tema principal?"

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [6]:
# Dividir o documento em chunks (exemplo simplificado)
chunks = [documento[i:i+512] for i in range(0, len(documento), 400)]  # Overlap de 112 tokens

In [7]:
respostas = []
for chunk in chunks:
    resposta = qa_pipeline(question=pergunta, context=chunk)
    respostas.append(resposta)

In [8]:
# Selecionar a resposta com maior score
melhor_resposta = max(respostas, key=lambda x: x['score'])
print(melhor_resposta)

{'score': 0.0002848559233825654, 'start': 247, 'end': 295, 'answer': 'achieving a\njust and inclusive energy transition'}
